# Higher-order functions

Higher-order functions (HOFs) are functions which receive or return other functions. HOFs show that functions can be regarded as _values_, much in the same way than integers, booleans, product values, sum values, etc. We will see that higher-order functions are essential modularity devices, and we will introduce the most common higher-order functions that operate over many different data structures.

## Functions as values

As we saw, functions are represented through methods in Scala (as in any other object-oriented language). But methods themselves can't be passed around and be returned by invocations. Therefore, the first thing to do in order to create HOFs in Scala is finding a way to reify methods. For instance, let's consider the following functions:

In [ ]:
def addOneM(number: Int): Int = 
    number + 1

def substractOneM(number: Int): Int = 
    number - 1 

We want to implement a function that receives an integer-to-integer function, such as `addOneM`and `substractOneM`, and call this function over a given number. We may want to write something like this:

where the first argument `int2int` attempt to represent any function that receives an integer and returns another integer. 

This code is not legal in Scala, but we can create a new class whose only method is the function that we want to pass around:

Now, we can implement the `call` HOF as follows: 

In [ ]:
// def call(int2int(number: Int): Int, number: Int): Int = ???

In order to use this HOF with the `addOneM` and `substractOneM` functions, we must create reified versions for them: 

In [ ]:
/*
def addOneM(number: Int): Int = 
    number + 1

def substractOneM(number: Int): Int = 
    number - 1 
*/

We call the `addOneV` and `substractOneV` function-values, i.e. functions represented as values. Now, we can use the `call` HOF as follows:

### Standard functions in Scala

The Scala programming language offers many facilities to work with functions as values. First, the standard library provides the following _generic_ types [`Function1`](https://www.scala-lang.org/api/current/scala/Function1.html), [`Function2`](https://www.scala-lang.org/api/current/scala/Function2.html), ...:

In [ ]:
/*abstract class FunctionInt2Int{
    def apply(number: Int): Int
}
*/
// up to Function22

Using these standard classes, we can create the `addOneV` function-value in a similar way: 

In [ ]:
/*
val addOneV: FunctionInt2Int = new FunctionInt2Int{
    def apply(number: Int): Int = 
        number + 1
}
*/

But we can do it more easily, since Scala also provides special syntax to declare function types and create functions (so-called _lambda expressions_):

And we can also profit from type inference:

Using these syntactic facilities we can write the `call` HOF more easily: 

In [ ]:
/*
def call(int2int: FunctionInt2Int, number: Int): Int = 
    int2int.apply(number)
*/

which we can use as follows:

We can even pass function-methods that are converted on the fly to function-values (this is the so-called _eta-expansion_):

### Currying

We can use a similar syntax for functions of two arguments. So, instead of writing the more verbose:

In [ ]:
// sum/2

we can create a lambda expressions for a `Function2` value in the following way:

or, exploiting type inference:

However, `Function2`, `Function3`, ... classes are not extrictly necessary, and sometimes we can get along with `Function1`. But, how can we create a function of two arguments with `Function1` alone? The trick is the following:

Note that brackets in `Int => (Int => Int)` are used for clarity, but are not needed. Basically, we created a function of one argument that returns another function of one argument. So, the expression: 

returns a function that can be applied again:

We can apply this strategy to functions of any number of arguments. This is called _currying_.

### Improved unit testing

In [ ]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

Using higher-order functions we can create test catalogues which are parameterised by the function to be tested. For instance, let's consider the following two alternative implementations:

In [ ]:
// Recursively

def sumR(list: List[Int]): Int = 
    list match {
        case Nil => 0 : Int
        case head :: tail => head + sumR(tail) : Int 
    }

In [ ]:
// With tail-recursion

def sumTR(list: List[Int]): Int = {

    def sumAux(acc: Int, list: List[Int]): Int = 
        list match {
            case Nil => acc : Int
            case head :: tail => sumAux(head + acc, tail) : Int 
        }
    
    sumAux(0, list)
}

Instead of creating ad-hoc test catalogues for each alternative, we can create a single one that receives the function to be tested as argument as follows:

In [ ]:
/*
object TestSumR extends FlatSpec with Matchers{
    "length" should "work" in {
        sumR(List()) shouldBe 0 
        sumR(List(1)) shouldBe 1 
        sumR(List(1,2,3,4)) shouldBe 10
    }
}
*/


Now, we can test the `sumR` and `sumTR` functions by reusing the same test catalogue:

## Functions compose

We can create new functions by composing other functions whose signatures match. This is great from a modularity perspective. For instance, the following function is implemented in a non-modular way:

In [ ]:
// isEvenLength


This function is somehow the combination of two more basic functions `length` and `isEven`:

but this is not reflected in the current implementation. How can we redefine the function `isEvenLength` from the functions `length` and `isEven`? We can use a HOF which is able to compose functions:

Then, we can redefine `isEvenLength` in a modular way from the `length` and `isEven` building blocks:

In [ ]:
/*
def isEvenLength: String => Boolean = 
    (s: String) => s.length % 2 == 0
    */

The HOF `compose` is actually defined by `Function1`: 

or using infix notation:

Note that a similar function to `compose`, called `andThen`, is also available: 

## HOFs as modularity devices

HOFs shine when the time comes to break monoliths. For instance, let's consider the following two functions:

In [ ]:
def sum(list: List[Int]): Int = 
    list match {
        case Nil => 0
        case head :: tail => head + sum(tail)
    }

In [ ]:
def multiply(list: List[Int]): Int = 
    ???

These functions clearly share a common logic; their only differences are the value which is returned when the list is empty, and the function used to combine numbers (`+` and `*`, respectively). We can abstract away these differences and arrive to a more generic function which encodes that common logic:

In [ ]:
/*
def sum(list: List[Int]): Int = 
    list match {
        case Nil => 0
        case head :: tail => head + sum(tail)
    }
*/

which allows us to re-define in a modular way the `sum` and `multiply` functions:

In [ ]:
/*
def sum(list: List[Int]): Int = 
    list match {
        case Nil => 0
        case head :: tail => head + sum(tail)
    }
*/

In [ ]:
/*
def multiply(list: List[Int]): Int = 
    list match {
        case Nil => 1
        case head :: tail => head * multiply(tail)
    }
*/

But we don't need to constrain ourselves to integers. In its generic version, the `combine` function is actually the `foldRight` higher-order function (for `List`'s):

In [ ]:
/*
def combine(list: List[Int])(nil: Int, cons: (Int, Int) => Int): Int = 
    list match {
        case Nil => nil
        case head :: tail => cons(head, combine(tail)(nil, cons))
    }
*/

The implementation of `sum` and `multiply` using `foldRight` is no more difficult:

In [ ]:
/*
def sum(list: List[Int]): Int = 
    combine(list)(0, (a, b) => a + b)
*/

In [ ]:
/*
def multiply(list: List[Int]): Int = 
    combine(list)(1, _ * _)
*/

although you may have noticed that we have to give extra type information in the invocations to `foldRight`. In fact, the following code doesn't compile. Check it yourself!

In [ ]:
/*
def multiply(list: List[Int]): Int = 
    foldRight(list)(1, (a, b) => a * b)
*/

In order to help the Scala compiler to infer the type parameters of the `foldRight` function, we need to change its signature a little bit: 

In [ ]:
/*
def foldRight[A, B](list: List[A])(nil: B, cons: (A, B) => B): B = 
    list match {
        case Nil => nil
        case head :: tail => cons(head, foldRight(tail)(nil, cons))
    }
*/

By splitting the second parameter list, we allow the Scala compiler to infer the type of `B`, before it analyses the type of the `cons` argument. Now this works:

In [ ]:
/*
def multiply(list: List[Int]): Int = 
    foldRight(list)(1, (a, b) => a * b)
*/

## The Hall of Fame of HOFs

Besides `compose`, `andThen` and `foldRight`, there are other famous higher-order functions which work great as modularity devices: 

In [ ]:
// foldLeft
// filter
// map
// flatMap

But most of these functions do not work only for `List`, they work also for `Option`, `Either[A, ?]`, and many other data structures. For instance, for `Option`:

In [ ]:
// fold
// filter
// map
// flatMap

In [ ]:
// fold
// filter
// map
// flatMap

The `fold` function is the catamorphism for `Option` and `Either[A, ?]`, in the same way than `foldRight` is the catamorphism for lists (we will see catamorphisms later on). 

## FoldLeft: a better loop

The `foldLeft` HOF is the functional way to implement common imperative algorithms. In the following implementation, its signature declares the initial value of a mutable variable, and the update function executed in every step of the loop:

In [ ]:
def foldLeft[A, B](list: List[A])(initial: B)(update: (B, A) => B): B = 
    ???

Also, the `foldLeft` function is typically used where a tail-recursive function is needed. Its recursive implementation is naturally tail-recursive:

In [ ]:
@annotation.tailrec
def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
    ???

Using `foldLeft` we can implement functions at a higher-level of abstraction, i.e. using constructs which are nearer to the problem-domain, without caring about mutable variables, tail-safe recursion, etc. Its use also leads to very concise (and readable!) implementations. For instance, these are one-liner implementations of many of the functions of the last notebook on recursive functions:

In [ ]:
def length[A](list: List[A]): Int = 
    ???

In [ ]:
def sum(list: List[Int]): Int = 
    ???

In [ ]:
def reverse[A](list: List[A]): List[A] = 
    ???